# Download data

In [ ]:
! gdown --id 1P7oWepNtdJu6Ko4jqdCytKgoRkIi7JmN
! gdown --id 1eavY3idLze3JZtc4SIKvZASAlgWs8CRj
! gdown --id 19XKrosZSuVJfqE-qUsqekVx4iROuV8mx

Downloading...
From: https://drive.google.com/uc?id=1P7oWepNtdJu6Ko4jqdCytKgoRkIi7JmN
To: /content/postag_test_gold.txt
100% 2.07k/2.07k [00:00<00:00, 1.76MB/s]
Downloading...
From: https://drive.google.com/uc?id=1eavY3idLze3JZtc4SIKvZASAlgWs8CRj
To: /content/postag_test.txt
100% 1.63k/1.63k [00:00<00:00, 3.06MB/s]
Downloading...
From: https://drive.google.com/uc?id=19XKrosZSuVJfqE-qUsqekVx4iROuV8mx
To: /content/postag_train.txt
100% 5.33k/5.33k [00:00<00:00, 9.14MB/s]


# Gán nhãn từ loại

## Hidden Markov Chain train data

### Xử lí dữ liệu đầu vào

In [ ]:
from decimal import *

In [ ]:
#lấy dữ liệu train
train_data = open("postag_train.txt", mode="r", encoding="utf-8").read().splitlines()

In [ ]:
#tạo set trạng thái hidden, các trạng thái này không lặp lại, dùng để tạo ma trận A và B
hidden_set = set()
#tạo set trạng thái quan sát được, tương tự các trạng thái quan sát này không lặp lại, dùng để tạo ma trận B
observer_set = set()
#list dữ liệu để train
train_observer = []
train_hidden = []

for i in range(0, len(train_data)):
    temp = train_data[i].strip().split(" ")
    train_observer.append([])
    train_hidden.append([])
    for j in temp:
        word = j.split("/")

        observer_set.add(word[0].lower())
        hidden_set.add(word[1])
        train_observer[i].append(word[0].lower())
        train_hidden[i].append(word[1])


In [ ]:
#chuyển từ set về list để lấy index
hidden_set = list(hidden_set)
hidden_set.sort()
# thêm trạng thái bắt đầu S vào đầu set của hidden_set
hidden_list = ["S"] + hidden_set
observer_list = list(observer_set)
observer_list.sort()
# theme cacs trạng thái unknow cho trạng thái quan sát được
observer_list.append("unk1")
observer_list.append("unk2")

In [ ]:
hidden_list

['S',
 'A',
 'C',
 'CH',
 'Cc',
 'E',
 'I',
 'L',
 'M',
 'N',
 'Nc',
 'Np',
 'P',
 'R',
 'T',
 'V']

In [ ]:
#các trạng thái này là không lặp lại
print(observer_list)

['!', '"', '(', ')', ',', '100.000', '1890', '1933', '1969', ':', ';', '?', 'anh_thư', 'australia', 'bao_gồm', 'bao_quát', 'bay_bổng', 'bà_rịa', 'bên', 'bình_dương', 'bước', 'bạn', 'bạn_bè', 'bạo_dạn', 'bản_chất', 'bảo', 'bức', 'cau_có', 'chao_ôi', 'che_chở', 'chiến_tích', 'cho', 'chuyện', 'chính', 'chúng_tôi', 'chăm_chỉ', 'chạm', 'chất_lượng', 'chị', 'chờ', 'chụp', 'chủ', 'chủ_tịch', 'chủ_động', 'chứng_minh', 'coi', 'con', 'con_người', 'covid-19', 'crater_oregon', 'các', 'cách', 'cách_mạng', 'cây', 'có', 'có_ích', 'cô', 'công_ty', 'công_việc', 'cùng', 'cúc', 'cũng', 'cơ_hội', 'cơ_thể', 'cơn', 'cạm_bẫy', 'cảm_giác', 'cảm_thấy', 'cầm_cự', 'cần', 'cố_gắng', 'của', 'danh_nhân', 'du_lịch', 'duy_nhất', 'dẫn', 'dễ_thương', 'dịch_bệnh', 'giàu', 'giáo_dục', 'giữa', 'gs.phạm', 'gương_mẫu', 'gắng_gượng', 'hiện', 'hoà', 'huyện', 'hàng_xóm', 'hành_động', 'hôm_nay', 'hạnh_phúc', 'hậu_quả', 'hắn', 'hết_sức', 'học', 'học_hành', 'học_hỏi', 'học_sinh', 'hồ', 'hồ_chí_minh', 'hội_nhập', 'khi', 'khiến', '

###Train Ma trận A (xác suất chuyển trạng thái) và Ma trận B (xác suất thể hiện trạng thái ẩn)

#### Train ma trận A

In [ ]:
list_map_pos_A = {}
matrix_A = []
for i in range(0, len(hidden_list)):
    #khởi tạo ma trận A là mảng 2 chiều của các số 1 (vì sử dụng laplace nên mặc định giá trị là 1), đồng thời thêm sẵn số 1 là tổng phần tử sau này
    matrix_A.append([Decimal(1)])

    #map dictionary các trạng thái hidden với các vị trí index cụ thể trong ma trận A tương ứng, để dễ dàng truy ra vị trí trong ma trận, 
    # ví dụ ('S', 'N'): (1, 0) thì trạng thái S chuyển sang N sẽ nằm ở vị trí 1,0 trong ma trận A tương ứng A[1,0]

    for j in range(1, len(hidden_list)):
        matrix_A[i].append(Decimal(1))
        list_map_pos_A.update({(hidden_list[i], hidden_list[j]): (i, j - 1)})

In [ ]:
list_map_pos_A

{('A', 'A'): (1, 0),
 ('A', 'C'): (1, 1),
 ('A', 'CH'): (1, 2),
 ('A', 'Cc'): (1, 3),
 ('A', 'E'): (1, 4),
 ('A', 'I'): (1, 5),
 ('A', 'L'): (1, 6),
 ('A', 'M'): (1, 7),
 ('A', 'N'): (1, 8),
 ('A', 'Nc'): (1, 9),
 ('A', 'Np'): (1, 10),
 ('A', 'P'): (1, 11),
 ('A', 'R'): (1, 12),
 ('A', 'T'): (1, 13),
 ('A', 'V'): (1, 14),
 ('C', 'A'): (2, 0),
 ('C', 'C'): (2, 1),
 ('C', 'CH'): (2, 2),
 ('C', 'Cc'): (2, 3),
 ('C', 'E'): (2, 4),
 ('C', 'I'): (2, 5),
 ('C', 'L'): (2, 6),
 ('C', 'M'): (2, 7),
 ('C', 'N'): (2, 8),
 ('C', 'Nc'): (2, 9),
 ('C', 'Np'): (2, 10),
 ('C', 'P'): (2, 11),
 ('C', 'R'): (2, 12),
 ('C', 'T'): (2, 13),
 ('C', 'V'): (2, 14),
 ('CH', 'A'): (3, 0),
 ('CH', 'C'): (3, 1),
 ('CH', 'CH'): (3, 2),
 ('CH', 'Cc'): (3, 3),
 ('CH', 'E'): (3, 4),
 ('CH', 'I'): (3, 5),
 ('CH', 'L'): (3, 6),
 ('CH', 'M'): (3, 7),
 ('CH', 'N'): (3, 8),
 ('CH', 'Nc'): (3, 9),
 ('CH', 'Np'): (3, 10),
 ('CH', 'P'): (3, 11),
 ('CH', 'R'): (3, 12),
 ('CH', 'T'): (3, 13),
 ('CH', 'V'): (3, 14),
 ('Cc', 'A'):

In [ ]:
for item in train_hidden:
    #Lấy vị trí S ứng với các trạng thái hidden trong ma trận A
    pos_S = list_map_pos_A[("S", item[0])]
    #tăng giá trị lên 1
    matrix_A[pos_S[0]][pos_S[1]] += 1
    #tăng giá trị cho các trạng thái còn lại
    for i in range(0, len(item) - 1):
        #lấy vị trí tương ứng với các trạng thái hidden
        cur_pos = list_map_pos_A[(item[i], item[i + 1])]
        matrix_A[cur_pos[0]][cur_pos[1]] += 1

In [ ]:
print(hidden_list)
for item in matrix_A:
  print(item)

['S', 'A', 'C', 'CH', 'Cc', 'E', 'I', 'L', 'M', 'N', 'Nc', 'Np', 'P', 'R', 'T', 'V']
[Decimal('1'), Decimal('3'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('2'), Decimal('1'), Decimal('31'), Decimal('1'), Decimal('8'), Decimal('8'), Decimal('1'), Decimal('1'), Decimal('3'), Decimal('1')]
[Decimal('6'), Decimal('3'), Decimal('13'), Decimal('3'), Decimal('11'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('7'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('3'), Decimal('1'), Decimal('5'), Decimal('1')]
[Decimal('2'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('3'), Decimal('1')]
[Decimal('8'), Decimal('2'), Decimal('2'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('3'), Decimal('12'), Decimal('1'), Decimal('4'), Decimal('3'), Decimal('1'), Decimal('1'), Decimal('9'), Decimal('1')]
[Decimal('4

In [ ]:
len_item = len(matrix_A[0])
#tính tổng giá trị phần tử trong hàng
for i in range(0, len(matrix_A)):
    #lưu vào phàn tử cuối (đã khởi tạo trước đó)
    matrix_A[i][len_item - 1] = sum(matrix_A[i][0:len_item - 1])
for item in matrix_A:
  print(item)


[Decimal('1'), Decimal('3'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('2'), Decimal('1'), Decimal('31'), Decimal('1'), Decimal('8'), Decimal('8'), Decimal('1'), Decimal('1'), Decimal('3'), Decimal('65')]
[Decimal('6'), Decimal('3'), Decimal('13'), Decimal('3'), Decimal('11'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('7'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('3'), Decimal('1'), Decimal('5'), Decimal('58')]
[Decimal('2'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('3'), Decimal('21')]
[Decimal('8'), Decimal('2'), Decimal('2'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('3'), Decimal('12'), Decimal('1'), Decimal('4'), Decimal('3'), Decimal('1'), Decimal('1'), Decimal('9'), Decimal('50')]
[Decimal('4'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal

In [ ]:
#tính xác suất bằng cách chia giá trị phần tử mảng cho giá trị cuối cùng trong 1 hàng ()
for i in range(0, len(matrix_A)):
    for j in range(0, len_item - 1):
        matrix_A[i][j] = matrix_A[i][j] / matrix_A[i][len_item - 1]
        
    #bỏ phần tử cuối chứa tổng (vì lúc này không cần tới nữa)
    matrix_A[i].pop()

In [ ]:
for item in matrix_A:
  print(item)

[Decimal('0.01538461538461538461538461538'), Decimal('0.04615384615384615384615384615'), Decimal('0.01538461538461538461538461538'), Decimal('0.01538461538461538461538461538'), Decimal('0.01538461538461538461538461538'), Decimal('0.03076923076923076923076923077'), Decimal('0.03076923076923076923076923077'), Decimal('0.01538461538461538461538461538'), Decimal('0.4769230769230769230769230769'), Decimal('0.01538461538461538461538461538'), Decimal('0.1230769230769230769230769231'), Decimal('0.1230769230769230769230769231'), Decimal('0.01538461538461538461538461538'), Decimal('0.01538461538461538461538461538'), Decimal('0.04615384615384615384615384615')]
[Decimal('0.1034482758620689655172413793'), Decimal('0.05172413793103448275862068966'), Decimal('0.2241379310344827586206896552'), Decimal('0.05172413793103448275862068966'), Decimal('0.1896551724137931034482758621'), Decimal('0.01724137931034482758620689655'), Decimal('0.01724137931034482758620689655'), Decimal('0.0172413793103448275862068

#### Train ma trận B

In [ ]:
# tương tự như ma trận A
list_map_pos_B = {}
matrix_B = []
#khởi tạo giá trị, map dictionary các phần tử
for i in range(1, len(hidden_list)):
    matrix_B.append([Decimal(1)])
    for j in range(0, len(observer_list)):
        matrix_B[i - 1].append(Decimal(1))
        list_map_pos_B.update({(hidden_list[i], observer_list[j]): (i - 1, j)})

#chạy mảng các phần tử chứa tất cả các từ trong train, tăng giá trị tương ứng
for i in range(0, len(train_observer)):
    for j in range(0, len(train_observer[i])):
        cur_pos = list_map_pos_B[(train_hidden[i][j], train_observer[i][j])]
        matrix_B[cur_pos[0]][cur_pos[1]] += 1

len_item = len(matrix_B[0])
#tính tổng
for i in range(0, len(matrix_B)):
    matrix_B[i][len_item - 1] = sum(matrix_B[i][0:len_item - 1])
for item in matrix_B:
  print(item)
#tính xác suất
for i in range(0, len(matrix_B)):
    for j in range(0, len_item - 1):
        matrix_B[i][j] = matrix_B[i][j] / matrix_B[i][len_item - 1]
    matrix_B[i].pop()

[Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('2'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decimal('1'), Decim

In [ ]:
for item in matrix_B:
  print(item)


[Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('0.002577319587628865979381443299'), Decimal('

In [ ]:
list_map_pos_B

{('A', '!'): (0, 0),
 ('A', '"'): (0, 1),
 ('A', '('): (0, 2),
 ('A', ')'): (0, 3),
 ('A', ','): (0, 4),
 ('A', '100.000'): (0, 5),
 ('A', '1890'): (0, 6),
 ('A', '1933'): (0, 7),
 ('A', '1969'): (0, 8),
 ('A', ':'): (0, 9),
 ('A', ';'): (0, 10),
 ('A', '?'): (0, 11),
 ('A', 'anh_thư'): (0, 12),
 ('A', 'australia'): (0, 13),
 ('A', 'bao_gồm'): (0, 14),
 ('A', 'bao_quát'): (0, 15),
 ('A', 'bay_bổng'): (0, 16),
 ('A', 'bà_rịa'): (0, 17),
 ('A', 'bên'): (0, 18),
 ('A', 'bình_dương'): (0, 19),
 ('A', 'bước'): (0, 20),
 ('A', 'bạn'): (0, 21),
 ('A', 'bạn_bè'): (0, 22),
 ('A', 'bạo_dạn'): (0, 23),
 ('A', 'bản_chất'): (0, 24),
 ('A', 'bảo'): (0, 25),
 ('A', 'bức'): (0, 26),
 ('A', 'cau_có'): (0, 27),
 ('A', 'chao_ôi'): (0, 28),
 ('A', 'che_chở'): (0, 29),
 ('A', 'chiến_tích'): (0, 30),
 ('A', 'cho'): (0, 31),
 ('A', 'chuyện'): (0, 32),
 ('A', 'chính'): (0, 33),
 ('A', 'chúng_tôi'): (0, 34),
 ('A', 'chăm_chỉ'): (0, 35),
 ('A', 'chạm'): (0, 36),
 ('A', 'chất_lượng'): (0, 37),
 ('A', 'chị'): (0,

## Thuật toán Viterbi

### Tính ma trận xác suất và ma trận poiter

In [ ]:
def calculate_matrix_prob_and_pointer(sentences, matrix_A, matrix_B):
    ##row ở đây là số hàng
    len_row = len(matrix_B)
    #col ở đây là số cột
    len_col = len(sentences)
    #khởi tạo 2 ma trận xác suất và ma trận pointer(để truy ngược sau này)
    matrix_probability = [[Decimal(0) for col in range(len_col)] for row in range(len_row)]
    matrix_pointer = [["" for col in range(len_col)] for row in range(len_row)]

    # Tính xác suất chuyển từ S(vị trí bắt đầu) sang từ đầu tiên
    for i in range(len_row):
        #lấy giá trị map dictionary của trạng thái ẩn của từ đầu tiên

        cur_pos_B = list_map_pos_B[(hidden_list[i + 1], sentences[0])]

        #lấy giá trị map dictionary của S chuyển sang trạng thái ẩn của từ đầu tiên

        cur_pos_A = list_map_pos_A[("S", hidden_list[i + 1])]

        #tính xác suất bằng = giá trị S chuyển sang trạng thái ẩn trong ma trận A * xác suất giá trị từ đầu tiên ứng với 
        #trạng thái ẩn trong ma trận B 

        matrix_probability[i][0] = matrix_A[cur_pos_A[0]][cur_pos_A[1]] * matrix_B[cur_pos_B[0]][cur_pos_B[1]]

        # vị trí từ đầu tiên trong matrix pointer là S tức ứng với nó trước đó là S

        matrix_pointer[i][0] = "S"
  #Chạy vòng lặp tính xác suất của từ tiếp theo, k là biến chạy trên từ
    for k in range(1, len(sentences)):
      #i là biến chạy trên 1 cột hàng sau
        for i in range(len_row):
            max_probability = 0
            #j là biến chạy trên 1 cột hàng trước
            for j in range(len_row):
              #ở đây i+1,j +1 trên hidden list vì không tính phần tử S (đã tính ở trên)

                #lấy vị trí chứa xác suất là là hidden i+1
                cur_pos_B = list_map_pos_B[(hidden_list[i + 1], sentences[k])]

                #lấy vị trí chuyển từ trạng thái hidden trước (j+1) sang trạng thái hidden i+1

                cur_pos_A = list_map_pos_A[(hidden_list[j + 1], hidden_list[i + 1])]
                # tính xác suất tại vị trí (k,i+1)= xác suất tại vị trí (j ,k-1) đã lưu * xác suất chuyển từ hidden j+1 sang i+1
                # * xác suất k là hidden i

                cur_prob = matrix_probability[j][k - 1] * matrix_A[cur_pos_A[0]][cur_pos_A[1]] * matrix_B[cur_pos_B[0]][cur_pos_B[1]]
                #nếu xác suất > xác suất max đã gán trước đó là gán lại
                if cur_prob > max_probability:
                    max_probability = cur_prob
                    matrix_probability[i][k] = max_probability
                    matrix_pointer[i][k] = hidden_list[j + 1]
    return matrix_probability, matrix_pointer

In [ ]:
sentences= "học_sinh đang cảm_thấy nhàm_chán"
sentences= sentences.split(" ")

matrix_probability, matrix_pointer= calculate_matrix_prob_and_pointer(sentences, matrix_A, matrix_B)
for item in matrix_probability:
  print(item)

[Decimal('0.00003965107057890563045202220459'), Decimal('0.000001995676463506802681674153672'), Decimal('1.654835509693581371796473373E-9'), Decimal('1.994707556079079249950188532E-12')]
[Decimal('0.0001373626373626373626373626373'), Decimal('8.863581362644682240036305229E-8'), Decimal('3.072162043575743044449128190E-10'), Decimal('2.547468457040611717666984872E-13')]
[Decimal('0.00004180602006688963210702341136'), Decimal('4.855701094318391140193801994E-7'), Decimal('1.215507591153880943673350719E-9'), Decimal('1.051557787715601561115045041E-12')]
[Decimal('0.00004485310607759587351424086117'), Decimal('6.077884362956353536024895015E-7'), Decimal('3.743884418432125785755286700E-10'), Decimal('6.446868236223929604170538531E-13')]
[Decimal('0.00004191993292810731502829595471'), Decimal('5.680420535406074285712640299E-7'), Decimal('1.031308893102265784948590172E-9'), Decimal('1.958214268754938835490220116E-12')]
[Decimal('0.00009295840111550081338600976065'), Decimal('8.99747231978433000

In [ ]:
for item in matrix_pointer:
  print(item)


['S', 'N', 'R', 'V']
['S', 'N', 'A', 'A']
['S', 'N', 'A', 'V']
['S', 'N', 'R', 'V']
['S', 'N', 'A', 'V']
['S', 'N', 'R', 'V']
['S', 'N', 'R', 'V']
['S', 'N', 'V', 'V']
['S', 'N', 'V', 'V']
['S', 'N', 'R', 'V']
['S', 'N', 'N', 'V']
['S', 'N', 'V', 'V']
['S', 'N', 'P', 'V']
['S', 'N', 'R', 'V']
['S', 'N', 'R', 'V']


### Truy ngược Viterbi

In [ ]:
def truy_nguoc_viterbi(matrix_probability, matrix_pointer):
    #số hàng
    len_row = len(matrix_probability)
    #số cột
    len_col = len(matrix_probability[0])
    #map dictionary vị trí tương ứng của trạng thái hidden với số thứ tự của nó trên mảng (theo chiều dọc)
    list_map_final_result = {}
    for i in range(1, len(hidden_list)):
        list_map_final_result.update({hidden_list[i]: i - 1})
    #print(list_map_final_result)
    max_final = 0
    final_pos = 1
    # tìm xác suất max ở cột cuối cùng
    for i in range(len_row):
        cur_val = matrix_probability[i][len_col - 1]
        if cur_val > max_final:
            final_pos = i + 1
            max_final = cur_val
    #appen trạng thái ẩn cuối cùng vào mảng 
    final_result = [hidden_list[final_pos]]
    #truy vết dựa vào trạng thái ẩn được lưu tại vị trí của trạng thái ẩn cuối cùng, những trạng thái này được lưu trong matrix pointer
    for i in range(0, len_col):
        final_result.append(matrix_pointer[list_map_final_result[final_result[i]]][len_col - i - 1])

    final_result.pop()
    final_result.reverse()
    return final_result

In [ ]:
truy_nguoc_viterbi(matrix_probability, matrix_pointer)
#tương ứng sentences= "công_ty phải có kỉ_luật , trật_tự rõ_ràng"

['N', 'R', 'V', 'N']

### Xử lí dữ liệu trả kết quả hoàn chỉnh

In [33]:
def viterbi_sentences(sentences, matrix_A, matrix_B):
    sentences = sentences.split(" ")
    copy_sentences = sentences.copy()
    unk_pos = "1"
    #kiểm tra xem có từ nào chưa có trong từ điển không, nếu có thì gán unk1, unk2,...
    for i in range(0, len(sentences)):
        sentences[i] = sentences[i].lower()
        if sentences[i] not in observer_set:
            sentences[i] = "unk" + unk_pos
            unk_pos = str(int(unk_pos) + 1)
    # tính ma trận xác suất pointer
    matrix_probability, matrix_pointer = calculate_matrix_prob_and_pointer(sentences, matrix_A, matrix_B)
   
    #truy ngước
    final_result = truy_nguoc_viterbi(matrix_probability, matrix_pointer)
    #trả kết quả tương ứng với câu
    for i in range(0, len(sentences)):
        final_result[i] = [copy_sentences[i]] + [final_result[i]]
    return final_result


In [34]:
sentences="học_sinh đang cảm_thấy nhàm_chán"
viterbi_sentences(sentences, matrix_A, matrix_B)

[['học_sinh', 'N'], ['đang', 'R'], ['cảm_thấy', 'V'], ['nhàm_chán', 'N']]

### Chạy trên bộ dữ liệu test chưa gán nhãn

In [ ]:
#import dữ liệu test
data = open("postag_test.txt", mode="r", encoding="utf-8").read().splitlines()
result = []
for item in data:
    item = viterbi_sentences(item, matrix_A, matrix_B)
    for i in range(len(item)):
        item[i] = "/".join(item[i])
    print(" ".join(item))
    result.append(" ".join(item))

[Decimal('0.00003965107057890563045202220459'), Decimal('9.978382317534013408370768361E-7'), Decimal('5.975326562000361254546965809E-10'), Decimal('1.114679753270022547933425012E-12'), Decimal('3.878080820133354686917123897E-15'), Decimal('8.171207079210747333638087142E-18'), Decimal('1.989990483884220969226210806E-20')]
[Decimal('0.0001373626373626373626373626373'), Decimal('5.276582631811672433971483238E-8'), Decimal('1.536081021787871522224564095E-10'), Decimal('1.838842450122316107985412009E-13'), Decimal('1.722407690260693244647257409E-16'), Decimal('2.358949662748340748103852538E-18'), Decimal('1.257882863178994355547735090E-21')]
[Decimal('0.00004180602006688963210702341136'), Decimal('2.427850547159195570096900998E-7'), Decimal('6.300072570804728714033213951E-10'), Decimal('1.175260174643393338581980719E-12'), Decimal('1.981517716708606228407236132E-14'), Decimal('2.362024487525000512084080335E-18'), Decimal('4.976840893447325493688864921E-21')]
[Decimal('0.00004485310607759587

In [ ]:
#ghi file
f = open("gan_nhan_hidden_markov.txt", "a")
for item in result:
  f.write(item+"\n")
f.close()

# So sánh với Hidden Markov sử dụng thư viện nltk

### Kết quả nhóm làm

In [27]:
#import tập đã gán nhãn bằng hidden markov
hidden_markov_result = open("gan_nhan_hidden_markov.txt", mode="r", encoding="utf-8").read().splitlines()
#import tập gold
gold_data = open("postag_test_gold.txt", mode="r", encoding="utf-8").read().splitlines()

array_gold_data = []
array_hidden_markov_result = []

#xử lí dữ liệu chuẩn
for i in range(0, len(gold_data)):
    temp = gold_data[i].strip().split(" ")
    temp1 = hidden_markov_result[i].strip().split(" ")

    array_hidden_markov_result.append([])
    array_gold_data.append([])

    for j in range(len(temp)):
        array_gold_data[i].append(temp[j].split("/"))
        array_hidden_markov_result[i].append(temp1[j].split("/"))
#giá trị max trên tập gold
maximum_result = 0
for i in range(len(array_gold_data)):
    for j in range(len(array_gold_data[i])):
        maximum_result += 1

result = 0
#chạy vòng lặp kiểm tra có trùng không
for i in range(len(array_hidden_markov_result)):
    for j in range(len(array_hidden_markov_result[i])):
        if array_hidden_markov_result[i][j][1] == array_gold_data[i][j][1]:
            result += 1



print("Xác suất hidden markov nhóm làm trên tập test với phương pháp smooth là Laplace ")
result / maximum_result

Xác suất hidden markov nhóm làm trên tập test với phương pháp smooth là Laplace 


0.8543689320388349

###Kết quả hidden markov của thư viện nltk

####import data

In [28]:
#import train data
data_train = open("postag_train.txt", mode="r", encoding="utf-8").read().splitlines()
train = []
#chuẩn hoá train data_train
for i in range(0, len(data_train)):
    temp = data_train[i].strip().split(" ")
    train.append([])
   
    for j in range(len(temp)):
        train[i].append((temp[j].split("/")[0], temp[j].split("/")[1]))

In [29]:
#import test data
data_test = open("postag_test_gold.txt", mode="r", encoding="utf-8").read().splitlines()
test = []
#chuẩn hoá train data test
for i in range(0, len(data_test)):
    temp = data_test[i].strip().split(" ")
    test.append([])
   
    for j in range(len(temp)):
        test[i].append((temp[j].split("/")[0], temp[j].split("/")[1]))


####chạy kiểm xác suất

In [30]:
import nltk
from nltk.probability import LaplaceProbDist, WittenBellProbDist
from nltk.tag.hmm import HiddenMarkovModelTrainer

#Sử dụng phương pháp LaplaceProbDist
trainer = HiddenMarkovModelTrainer()
tagger=trainer.train_supervised(train,estimator=LaplaceProbDist)
print("Xác suất hidden markov chạy trên bộ test với hidden markov chain sử dụng phương pháp smooth là Laplace: ")
tagger.evaluate(test)

Xác suất hidden markov chạy trên bộ test với hidden markov chain sử dụng phương pháp smooth là Laplace: 


0.8543689320388349

In [31]:
#Sử dụng phương pháp WittenBellProbDist
trainer = HiddenMarkovModelTrainer()
tagger=trainer.train_supervised(train,estimator=WittenBellProbDist)
print("Xác suất hidden markov chạy trên bộ test với hidden markov chain sử dụng phương pháp smooth là WittenBell: ")
tagger.evaluate(test)

Xác suất hidden markov chạy trên bộ test với hidden markov chain sử dụng phương pháp smooth là WittenBell: 


0.9368932038834952